# 01_数据与实验设计报告

**项目名称**: SASRec.pytorch - 基于Transformer的序列推荐系统  
**版本**: v1.0  
**创建日期**: 2024-01-10  

---

## 目录

1. [项目背景](#1-项目背景)  
2. [数据集介绍](#2-数据集介绍)  
3. [数据格式](#3-数据格式)  
4. [数据预处理](#4-数据预处理)  
5. [数据分析](#5-数据分析)  
6. [实验设置](#6-实验设置)  
7. [基准模型](#7-基准模型)  

---

## 1. 项目背景

### 1.1 序列推荐问题定义

序列推荐（Sequential Recommendation）旨在根据用户的历史行为序列，预测用户下一个可能交互的物品。

**问题形式化**：  
给定用户 $u$ 的历史交互序列 $S_u = [i_1, i_2, ..., i_n]$，预测下一个交互物品 $i_{n+1}$。

### 1.2 研究动机

传统推荐系统（如协同过滤）假设用户兴趣是静态的，忽略了：
- 用户兴趣随时间的演变
- 物品之间的时序依赖关系
- 用户行为的上下文相关性

序列推荐通过建模用户行为的时间动态性，能够更准确地捕捉用户当前的需求。

### 1.3 技术路线

本项目实现以下三种模型：

| 模型 | 特点 | 核心创新 |
|------|------|----------|
| SASRec | 基于自注意力机制 | 捕捉序列长期依赖 |
| TiSASRec | 加入时间间隔信息 | 学习用户兴趣演变 |
| SASRec/TiSASRec + mHC | 流形约束超连接 | 增强训练稳定性 |

## 2. 数据集介绍

### 2.1 MovieLens 1M数据集

MovieLens 1M是推荐系统领域广泛使用的基准数据集，包含用户在MovieLens网站上的电影评分记录。

**数据集统计信息**：

| 统计项 | 数值 |
|--------|------|
| 用户数量 | 6,040 |
| 物品数量 | 3,952 |
| 交互数量 | 1,000,209 |
| 时间跨度 | 2000-2003年 |
| 稀疏度 | 95.81% |

### 2.2 数据来源

原始数据集来源：https://grouplens.org/datasets/movielens/

**引用**：  
F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19.

## 3. 数据格式

### 3.1 原始数据格式

项目使用的数据格式为：

```
用户ID 物品ID 时间戳
```

**字段说明**：

| 字段 | 类型 | 说明 |
|------|------|------|
| 用户ID | int | 用户唯一标识符 |
| 物品ID | int | 物品（电影）唯一标识符 |
| 时间戳 | int | Unix时间戳（秒） |

### 3.2 示例数据

以下代码展示示例数据的加载：

In [ ]:
# 加载示例数据
import pandas as pd

data_path = "../data/sample_data.txt"
columns = ["user_id", "item_id", "timestamp"]

df = pd.read_csv(data_path, sep=" ", header=None, names=columns)

print("示例数据前10行：")
print(df.head(10))
print(f"\n数据规模：{len(df)} 条交互记录")

## 4. 数据预处理

### 4.1 预处理流程

数据预处理包括以下步骤：

```
原始数据
    ↓
按用户分组
    ↓
按时间戳排序
    ↓
划分训练/验证/测试集
    ↓
构建字典
    ↓
生成序列数据
```

### 4.2 训练/验证/测试集划分

采用时序划分策略：

| 集合 | 比例 | 说明 |
|------|------|------|
| 训练集 | 80% | 每个用户的前N-2个物品 |
| 验证集 | 10% | 每个用户的第N-1个物品 |
| 测试集 | 10% | 每个用户的最后1个物品 |

### 4.3 时间间隔计算

对于TiSASRec，需要计算相邻物品之间的时间间隔：

In [ ]:
# 按用户分组并排序
user_groups = df.groupby("user_id")

# 划分训练/验证/测试集（按时间顺序）
def split_by_time(groups, train_ratio=0.8, valid_ratio=0.1):
    """按时间顺序划分数据集"""
    train_data = {}
    valid_data = {}
    test_data = {}
    
    for user_id, items in groups:
        items = items.sort_values("timestamp")
        item_ids = items["item_id"].tolist()
        timestamps = items["timestamp"].tolist()
        
        n = len(item_ids)
        train_end = int(n * train_ratio)
        valid_end = int(n * (train_ratio + valid_ratio))
        
        train_data[user_id] = item_ids[:train_end]
        valid_data[user_id] = [item_ids[valid_end - 1]]  # 验证集：第N-1个
        test_data[user_id] = [item_ids[-1]]  # 测试集：最后一个
    
    return train_data, valid_data, test_data

train_set, valid_set, test_set = split_by_time(user_groups)

print(f"训练集用户数：{len(train_set)}")
print(f"验证集用户数：{len(valid_set)}")
print(f"测试集用户数：{len(test_set)}")

## 5. 数据分析

### 5.1 序列长度分布

In [ ]:
import matplotlib.pyplot as plt

# 计算每个用户的序列长度
seq_lengths = [len(items) for items in train_set.values()]

# 可视化序列长度分布
plt.figure(figsize=(10, 4))
plt.hist(seq_lengths, bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('序列长度')
plt.ylabel('用户数量')
plt.title('用户交互序列长度分布')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n序列长度统计：")
print(f"最小长度：{min(seq_lengths)}")
print(f"最大长度：{max(seq_lengths)}")
print(f"平均长度：{sum(seq_lengths)/len(seq_lengths):.1f}")

### 5.2 交互频率分布

分析物品的交互频率分布：

In [ ]:
# 计算物品交互频率
item_counts = df["item_id"].value_counts()

plt.figure(figsize=(10, 4))
plt.hist(item_counts.values, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('交互次数')
plt.ylabel('物品数量')
plt.title('物品交互频率分布')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n物品统计：")
print(f"总物品数：{len(item_counts)}")
print(f"最热门物品交互次数：{item_counts.max()}")
print(f"最冷门物品交互次数：{item_counts.min()}")

### 5.3 时间间隔分析

分析相邻交互之间的时间间隔：

In [ ]:
# 计算时间间隔（转换为天数）
time_diffs = []
for user_id, items in user_groups:
    items = items.sort_values("timestamp")
    timestamps = items["timestamp"].tolist()
    for i in range(1, len(timestamps)):
        diff_days = (timestamps[i] - timestamps[i-1]) / (24*3600)
        time_diffs.append(diff_days)

# 可视化时间间隔分布
plt.figure(figsize=(10, 4))
plt.hist(time_diffs, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('时间间隔（天）')
plt.ylabel('次数')
plt.title('相邻交互时间间隔分布')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n时间间隔统计：")
print(f"平均间隔：{sum(time_diffs)/len(time_diffs):.1f} 天")
print(f"中位数间隔：{sorted(time_diffs)[len(time_diffs)//2]:.1f} 天")

## 6. 实验设置

### 6.1 模型超参数

| 参数 | 默认值 | 说明 |
|------|--------|------|
| hidden_units | 50 | 嵌入维度 |
| num_blocks | 2 | Transformer层数 |
| num_heads | 2 | 注意力头数 |
| maxlen | 200 | 最大序列长度 |
| dropout_rate | 0.2 | Dropout比例 |
| batch_size | 128 | 批次大小 |
| lr | 0.001 | 学习率 |
| num_epochs | 300 | 训练轮数 |

### 6.2 评估指标

采用以下两个评估指标：

- **HR@K（命中率）**：推荐列表中是否包含目标物品
- **NDCG@K（归一化折损累积增益）**：推荐列表的排序质量

默认使用 K=10。

### 6.3 训练配置

| 配置项 | 值 |
|--------|-----|
| 优化器 | Adam |
| 损失函数 | BCEWithLogitsLoss |
| 学习率衰减 | StepLR (step=1000, gamma=0.98) |
| 早停 | patience=3 |
| 正则化 | l2_emb=0.0 |

## 7. 基准模型

### 7.1 基准模型列表

本项目实现以下基准模型用于对比：

| 模型 | 说明 |
|------|------|
| SASRec | 基准自注意力序列推荐 |
| SASRec + mHC | 加入流形约束超连接 |
| TiSASRec | 时序感知自注意力推荐 |
| TiSASRec + mHC | 完整模型（默认配置） |

### 7.2 预期性能

根据文献和初步实验，预期性能范围：

| 模型 | NDCG@10 | HR@10 |
|------|---------|-------|
| SASRec | 0.40-0.45 | 0.70-0.75 |
| TiSASRec | 0.42-0.47 | 0.73-0.78 |
| + mHC | +2-4% | +2-4% |

---

**下一章**: [02_模型架构与实现报告.ipynb →](./02_模型架构与实现报告.ipynb)